# Natural_Language-2-SQL
#### Dependencies/Configuration
##### (Ctrl/Enter to render Markdown)

In [1]:
# This code block imports necessary libraries and modules for connecting to databases, processing data, 
# and utilizing the LangChain framework for SQL generation from natural language. It also sets up 
# environment variables and defines a function for Markdown display in Jupyter notebooks.
import os
import pandas as pd
import pyodbc
from common.prompts import MSSQL_AGENT_PREFIX
from IPython.display import Markdown, HTML, display  
from dotenv import load_dotenv
import logging
import sys

# dependencies to use EntraID/managed identity authentication
import struct
from azure.identity import DefaultAzureCredential, get_bearer_token_provider,InteractiveBrowserCredential

# LangChain dependencies
from langchain_openai import AzureChatOpenAI
from langchain_community.agent_toolkits import create_sql_agent, SQLDatabaseToolkit
from langchain.agents import initialize_agent
from langchain.agents import Tool
from langchain.agents import create_sql_agent  # No longer in agent_toolkits
from langchain.sql_database import SQLDatabase  # Toolkit is replaced with SQLDatabase
from langchain.agents import AgentExecutor
from langchain.callbacks.manager import CallbackManager
from langchain.memory import ConversationBufferMemory

# Database dependencies
from sqlalchemy import create_engine, inspect, text, event
from sqlalchemy.engine import URL

# Flag that controls whether to show query execution steps
show_query_execution_steps = True #False #True

# Function to print comments using markdown
def printmd(string):
    display(Markdown(string))

# load secrets from .env file
load_dotenv("credentials.env")

# validate that the environment variables are set
if "AZURE_OPENAI_API_VERSION" not in os.environ:
    raise ValueError("Please set the AZURE_OPENAI_API_VERSION environment variable in the credentials.env file.")
if "AZURE_OPENAI_API_VERSION" not in os.environ:
    raise ValueError("Please set the AZURE_OPENAI_API_VERSION environment variable in the credentials.env file.")
else:
    app_env = os.environ["AZURE_OPENAI_API_VERSION"]


# This code block sets up logging for the application. 
# It defines a custom log format that includes the timestamp, # logger name, log level, and the log message. # 

# Define a custom log format
log_format = '%(asctime)s - %(name)s - %(levelname)s - %(message)s'

# Python Logging Levels:
# DEBUG: Detailed information, typically of interest only when diagnosing problems.
# INFO: Confirmation that things are working as expected.
# WARNING: An indication that something unexpected happened, or indicative of some problem in the near future (e.g., ‘disk space low’). The software is still working as expected.
# ERROR: Due to a more serious problem, the software has not been able to perform some function.
# CRITICAL: A very serious error, indicating that the program itself may be unable to continue running.

# logging.basicConfig(level=logging.DEBUG)
# logging.basicConfig(level=logging.DEBUG, format=log_format)
logging.basicConfig(level=logging.INFO, format=log_format)

# Create a logger
logger = logging.getLogger('langchain')

printmd("#### Logger created...")

printmd("##### Depedencies loaded...")

#### Logger created...

##### Depedencies loaded...

#### Enable Logging

In [ ]:
# # This code block sets up logging for the application. 
# # It defines a custom log format that includes the timestamp, # logger name, log level, and the log message. # 

# # Define a custom log format
# log_format = '%(asctime)s - %(name)s - %(levelname)s - %(message)s'

# # Python Logging Levels:
# # DEBUG: Detailed information, typically of interest only when diagnosing problems.
# # INFO: Confirmation that things are working as expected.
# # WARNING: An indication that something unexpected happened, or indicative of some problem in the near future (e.g., ‘disk space low’). The software is still working as expected.
# # ERROR: Due to a more serious problem, the software has not been able to perform some function.
# # CRITICAL: A very serious error, indicating that the program itself may be unable to continue running.

# # logging.basicConfig(level=logging.DEBUG)
# # logging.basicConfig(level=logging.DEBUG, format=log_format)
# logging.basicConfig(level=logging.INFO, format=log_format)

# # Create a logger
# logger = logging.getLogger('langchain')

# printmd("#### Logger created...")

#### Determine if we will use DefaultAzureCredential object for DB and AI connections

In [ ]:
# check our environmental variables to see if we have defined a SQL username and an OpenAI API key
# we will use the presence or absence of these variables to set boolean values which can be used later
# when we create the SQL DB engine/connection and the connection to our LLM
# if these values due not exist or are empty strings, the DefaultAzureCredential object will be used for these connections

if "SQL_SERVER_USERNAME" not in os.environ: 
    sql_use_default_credential=True
else:
    if os.environ["SQL_SERVER_USERNAME"] == "":
        sql_use_default_credential=True
    else:
        sql_use_default_credential=False

if "AZURE_OPENAI_API_KEY" not in os.environ: 
    llm_use_default_credential=True
else:
    if os.environ["AZURE_OPENAI_API_KEY"] == "":
        llm_use_default_credential=True
    else:
        llm_use_default_credential=False

if sql_use_default_credential:
    printmd ("##### SQL_SERVER_USERNAME not defined or empty string.  We will attempt to use DefaultAzureCredential for our SQL connection.")
else:
    printmd ("##### SQL_SERVER_USERNAME found.  We will use username and password authentication for our SQL connection")

if llm_use_default_credential:
    printmd ("##### AZURE_OPENAI_API_KEY not defined or empty string.  We will attempt to use DefaultAzureCredential for our LLM connection.") 
else:
    printmd ("##### AZURE_OPENAI_API_KEY found.  We will use this API key for our LLM connection.")

##### Connect to Database and extract metadata -- Using SqlAlchemy and pyodbc   

In [ ]:
# Connect to Azure SQL Database using SQLAlchemy and pyodbc. 
# pyodbc is a Python DB API 2 module for ODBC.

if sql_use_default_credential:
    printmd ("#### Creating DB Connection with DefaultAzureCredential object.")
    
    sql_server = os.environ["SQL_SERVER_NAME"]
    sql_database = os.environ["SQL_SERVER_DATABASE"]

    SQL_COPT_SS_ACCESS_TOKEN = 1256  # Connection option for access tokens, as defined in msodbcsql.h
    TOKEN_URL = "https://database.windows.net/"  # The token URL for any Azure SQL database
    FABRIC_SCOPE = "https://fabric.microsoft.com/.default" 
    connection_string = "mssql+pyodbc://"+sql_server+"/"+sql_database+"?driver=ODBC+Driver+18+for+SQL+Server"

    engine = create_engine(connection_string)

    azure_credentials = DefaultAzureCredential()

    @event.listens_for(engine, "do_connect")
    def provide_token(dialect, conn_rec, cargs, cparams):
        # remove the "Trusted_Connection" parameter that SQLAlchemy adds
        cargs[0] = cargs[0].replace(";Trusted_Connection=Yes", "")

        # create token credential
        raw_token = azure_credentials.get_token(TOKEN_URL, scopes=[FABRIC_SCOPE]).token.encode("utf-16-le")
        token_struct = struct.pack(f"<I{len(raw_token)}s", len(raw_token), raw_token)

        # apply it to keyword arguments
        cparams["attrs_before"] = {SQL_COPT_SS_ACCESS_TOKEN: token_struct}

else:
    printmd ("#### Creating DB connection with username and password authentication.")

    # Configuration for the database connection
    db_config = {
        'drivername': 'mssql+pyodbc',
        'host': os.environ["SQL_SERVER_NAME"],
        'username': os.environ["SQL_SERVER_USERNAME"],
        'password': os.environ["SQL_SERVER_PASSWORD"],
        'port': 1433,
        'database': os.environ["SQL_SERVER_DATABASE"],
        'query': {'driver': 'ODBC Driver 18 for SQL Server'},
    }


    # Create a URL object for connecting to the database
    db_url = URL.create(**db_config)

    #####################################################
    ## Following Blocks Test the Database Connection:
    # Connect to the Azure SQL Database using the URL string
    engine = create_engine(db_url)

# # inspector = inspect(engine)

# # # Retrieve the list of tables
# # tables = inspector.get_table_names()

# # # Access columns and other schema details
# # for table_name in tables:
# #     columns = inspector.get_columns(table_name)
# #     for column in columns:
# #         print(f"Table: {table_name}, Column: {column['name']}, Type: {column['type']}")
    

# Test the connection using the SQLAlchemy 2.0 execution style
with engine.connect() as conn:
    try:
        # Use the text() construct for safer SQL execution
        result = conn.execute(text("SELECT @@VERSION"))
        version = result.fetchone()
        print("Connection successful!")
        print(version)
    except Exception as e:
        print(e)
        sys.exit(1)  # Exit the application with a non-zero status code indicating an error
## Test Complete
#####################################################

# Create instance of the SQLDatabase class. 
try:
    # Under the hood,SqlAlchemy's create_engine is used to connect to DB's URI.
    # Once connected, SqlAlchemy's MetaData and inspector object are used to intrpspect the DB's schema, 
    # extracting information about tables, columns, and relationships.
    # Using SqlAlchemy's ORM, it can map tables to Python classes and provides methods to query data, 
    # fetch results, and interact with the databae. 
    #SqlAlchemy an ORM. It's phasing out the toolkit, so we will use the SQLDatabase class instead
    #db = SQLDatabase(engine) #.from_uri(db_url)
    db = SQLDatabase(engine)

    #db = SQLDatabase.from_uri(db_url, include_tables=filtered_tables)
except Exception as e:
    print(f"An error occurred while connecting to the database: {e}")
    db = None  # Optionally, set db to None or handle the error as needed
    sys.exit(1)  # Exit the application with a non-zero status code indicating an error

# Test the connection to the database by fetching the table names
if show_query_execution_steps:
    print("Found following tables:", db.get_usable_table_names())

printmd(f"#### Connected to Database!")

##### Initialize AzureOpenAI and Sql Agent

In [ ]:
# Initialize instance of AzureChatOpenAI 
if llm_use_default_credential:
    printmd ("#### Creating LLM with DefaultAzureCredential object.") 
    token_provider = get_bearer_token_provider(
        DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default"
    )

    llm = AzureChatOpenAI(
        deployment_name=os.environ["GPT4_DEPLOYMENT_NAME"], temperature=0.2, max_tokens=2000, api_version=os.environ["AZURE_OPENAI_API_VERSION"],
        azure_ad_token_provider=token_provider
    )
else:
    printmd ("#### Creating LLM using API key.") 
    llm = AzureChatOpenAI(deployment_name=os.environ["GPT4_DEPLOYMENT_NAME"], temperature=0.2, max_tokens=2000, api_version=os.environ["AZURE_OPENAI_API_VERSION"])

printmd(f"## LLM Created: {llm.deployment_name}")

system_prompt = """
    You are an SQL expert with deep expertise in generating accurate and efficient SQL queries from natural language.
    Your goal is to understand the user's intent expressed in natural language and translate it into a correct and optimized SQL query that can be executed against the database to retrieve the desired information.
    DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.
    If the question does not seem related to the database, just return "I don't know" as the answer.
"""
 
# SQLDatabaseToolkit is a utility for interacting with a SQL database using a language model (LLM).
# It facilitate the integration between the database and the language model, enabling more sophisticated
# query generation, execution, and result handling.
# Parameters in the SQLDatabase class and the LLM.
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

# Create the SqlAgent
# SqlAgent interacts with directly with the SQL database. 
# It leverages the LLM to generate SQL queries from natural language input
# and then execute these queries on the connected database. 
# Note how the SqlAgent leverges the SQLDatabaseToolkit and the language model (LLM). 
try:
    agent_executor = create_sql_agent(
        prefix=MSSQL_AGENT_PREFIX,
        llm=llm,
        temperature=0.1,
        toolkit=toolkit,
        top_k=30,
        agent_type="openai-tools",
        verbose=show_query_execution_steps, #True,
        agent_executor_kwargs={"return_intermediate_steps": True},
        system_prompt=system_prompt  # Step 2: Pass the system prompt to the AgentExecutor
    )
except Exception as e:
    print(f"An error occurred while creating the agent_executor: {str(e)}")
    agent_executor = None  # Set agent_executor to None or handle it as needed
    sys.exit(1)  # Exit the application with a non-zero status code indicating an error

# Continue with the rest of your code, checking if agent_executor was created successfully
if agent_executor:
    printmd(f"##### SqlAgent Created")

# Langchain agents use expert/tools. List the tools for SQL Agent
agent_executor.tools

printmd(f"#### Complete")

#### Prompt Engineering

In [ ]:
# prompting directives at application level. They are sent to the LLM along with the user prompt.
prompting_hints = """
  
  loanstable: The loans table contains information about the loan applicant information
  Facts you need to know:
    - Never fetch more than 500 rows
    
  """
# Prompt for the question:
prompt = """
	Please produce a table listing Average credit score of applicants by their ownership type
"""

# Concatenate the prompting hints and the prompt
QUESTION = f"{prompting_hints} {prompt}"

if show_query_execution_steps:
  printmd(f"{prompting_hints}")

printmd(f"**User Prompt:** {prompt}")

#### Execute SQL Opeation

In [ ]:
# Invoke the SQL agent with the natural language question
# The agent will generate a SQL query based on the input question   
# and execute the query against the connected database.
# The response contains the query result and other relevant information.
# The response is a dictionary with keys such as 'query', 'result', 'intermediate_steps', and 'execution_time'.
try:
    response = agent_executor.invoke(QUESTION) 
except Exception as e:
    response = str(e)

# Advanced logging block
# Block iterates through the intermediate steps of the response 
# and prints the action and observation for each step.
# This is a common pattern for displaying detailed information based on 
# the content of data structures and conditional flag in Python
# Importantly, the block reveals the agent's decision-making process.
if show_query_execution_steps: 
    # Iterate through each step in the 'intermediate_steps' list from the response
    for step in response['intermediate_steps']:
         # Check if the current step is a dictionary
        if isinstance(step, dict):
            # Extract 'action' and 'observation' from the step, defaulting to empty string if not found
            action = step.get('action', '')
            observation = step.get('observation', '')
        # Check if the current step is a tuple (assumed to be a pair of action and observation)
        elif isinstance(step, tuple):
            # Unpack the tuple directly into action and observation variables
            action, observation = step 
        else:
            # Handle unexpected step types by printing a warning and skipping the current iteration
            print(f"Unexpected step type: {type(step)}")
            continue  # Skip this step if it's neither dict nor tuple
        # Print the action and observation for the current step
        print(f"Action: {action}\nObservation: {observation}\n")
# else:
#     # If the flag to show details is not set, indicate that there are no details to show
#     print("No details to show")

#### Final Answer

In [ ]:
import re

match = re.search(r'Final Answer:.*?(?=\n\n|$)', response["output"], re.DOTALL)
final_answer = match.group(0) if match else "Final Answer not found."

output_without_final_answer = re.sub(r'Final Answer:.*?(?=\n\n|$)', '', response["output"], flags=re.DOTALL).strip()

# printmd(f"**Response:** {response["output"]}")
printmd(f" User Prompt: {prompt}")
printmd(f" {final_answer}")
printmd(f" {output_without_final_answer}")


printmd(f"## Complete")